

In this notebook we will add the information from the cell segmentation to the transcripts.

With the MERSCOPE output we got a CSV called `detected_transcripts.csv` with the genes, x, y and z postitions.

In addition to that, we have quanitfied the auxiliary stains and merged the two transcripts files

We will import that file and check for each transcript, if it is in a cell, as detected by `cellpose`

In [1]:
!head ../auxiliary/all_transcripts.csv | column -t -s, | cat

global_x   global_y   global_z  gene
3822.614   134.60324  4.0       Pycr1
3865.5085  89.455734  5.0       Slc7a11
3972.9065  163.4522   6.0       Cxcr3
3968.8958  33.91168   0.0       Ldhb
3814.5425  132.51677  0.0       Blank-42
3664.2427  184.7953   4.0       Slc2a2
3797.8604  28.676212  0.0       Ets1
3687.2493  154.53114  1.0       Cxcr3
3635.9307  184.15593  2.0       Cxcr3


There are a few caveats to this. We need to adjust the scace / pixel ratio.  The information is stored in `micron_to_mosaic_pixel_transform.csv`, which is also found in the MERSCOPE output. There once again, is no good documentation about the file, but I assusme that the first two values are the scaling for x and y and the third is for z

In [2]:
!bat ../MERLIN_output/pictures/micron_to_mosaic_pixel_transform.csv

───────┬────────────────────────────────────────────────────────────────────────
       │ File: ../MERLIN_output/pictures/micron_to_mosaic_pixel_transform.csv
───────┼────────────────────────────────────────────────────────────────────────
   1   │ 9.259414672851562500e+00 0.000000000000000000e+00 -0.000000000000000000
       │ e+00
   2   │ 0.000000000000000000e+00 9.259401321411132812e+00 -0.000000000000000000
       │ e+00
   3   │ 0.000000000000000000e+00 0.000000000000000000e+00 1.000000000000000000e
       │ +00
───────┴────────────────────────────────────────────────────────────────────────


However, we need to keep in mind that we downsampled the image in order to run cellpose. We need to take that into account. We will calculte the scaling factors first.

In [ ]:
# import the libraries
import tifffile
import pandas as pd
import numpy as np

In [ ]:
# # the get the size of the original image, we can take any of the images
# # they are all the same size
# size_original_image = tifffile.imread("../Merlin_output/images/mosaic_Cellbound2_z0.tif").shape
# size_original_image

In [ ]:
# # rescaled image
# size_rescaled_image = tifffile.imread("image/full_stack.tif").shape
# # since this is the stacked image, we dont need to keep all dimensions.
# size_rescaled_image = size_rescaled_image[2:4]
# size_rescaled_image

In [ ]:
# mmpt = pd.read_csv(
#     "../Merlin_output/images/micron_to_mosaic_pixel_transform.csv", 
#     header=None, 
#     sep=' '
# )
# mmpt = np.array(mmpt)
# mmpt

There is obivously no documentation about the "transformation matrix" but but inspecting the values I assume the follwoing:
- The diagonal in the scaling.
- the values [0,2] and [1,2] are the offset?

In [ ]:
# scale_y = size_original_image[0] / size_rescaled_image[0]
# scale_x = size_original_image[1] / size_rescaled_image[1]
# scale_y, scale_x

# rescale = np.array([scale_x, scale_y, 1])
# rescale

Now, that we have the scaling factors, we can import the transcripts table.

In [ ]:
transcripts = pd.read_csv("../auxiliary/all_transcripts.csv")
transcripts

,Unnamed: 0,barcode_id,global_x,global_y,global_z,x,y,fov,gene,transcript_id
0,76052,29,4811.931152,133.306046,1.0,106.918274,1234.315186,0,Ctcf,ENSMUST00000005841
1,578964,170,4873.683594,128.109299,3.0,678.702148,1186.197266,0,Ets1,ENSMUST00000034534
2,663018,194,4963.922363,117.802322,0.0,1514.247681,1090.762207,0,Prdm1,ENSMUST00000039174
3,802336,241,4892.742188,192.105209,0.0,855.170471,1778.751831,0,Cxcr3,ENSMUST00000056614
4,805053,241,4972.506836,186.719177,3.0,1593.731567,1728.881226,0,Cxcr3,ENSMUST00000056614
...,...,...,...,...,...,...,...,...,...,...
260479,724991,221,2836.630859,9747.528320,0.0,337.099091,1358.897705,1467,Cd74,ENSMUST00000050487
260480,977945,298,2926.006836,9729.467773,0.0,1164.655518,1191.672241,1467,Arnt,ENSMUST00000090804
260481,1454933,435,2815.286865,9769.478516,0.0,139.470963,1562.134277,1467,Aldh18a1,ENSMUST00000176939
260482,1577062,477,2966.376953,9744.248047,0.0,1538.451660,1328.524292,1467,Hmgcs1,ENSMUST00000224188


In [ ]:
# diagnoal of the matrix for scaling
# np.diag(mmpt)

In [ ]:
# third col for offset,
# but the dont want to add a value to the z axis...
# np.append(mmpt[0:2,2], 0)

Let's test if it works with a few selected rows

In [ ]:
# from numpy.linalg import inv
# idx = np.arange(0,20)
# points = np.array([
#     transcripts.global_x[idx], 
#     transcripts.global_y[idx], 
#     transcripts.global_z[idx]]
# )
# points = np.round(np.diag(mmpt) * points.T + np.append(mmpt[0:2,2], 0)).astype(int)
# points

It seems to be working. So lets comment all the cells above and warp everyting into a function

In [ ]:
def create_rescaling_function():
    """
    Here we create a rescaling function that takes three argument: x, y, z.
    These are the coordinates of out point. We will apply the transformation
    defined in 'micron_to_mosaic_pixel_transform' to convert the micron to pixel
    and then rescale it with the same scaling factor used in script '01'
    """
    # the get the size of the original image, we can take any of the images
    # they are all the same size
    size_original_image = tifffile.imread("../Merlin_output/images/mosaic_Cellbound2_z0.tif").shape
    
    # rescaled image
    size_rescaled_image = tifffile.imread("image/full_stack.tif").shape
    # since this is the stacked image, we dont need to keep all dimensions.
    size_rescaled_image = size_rescaled_image[2:4]
    
    scale_y = size_original_image[0] / size_rescaled_image[0]
    scale_x = size_original_image[1] / size_rescaled_image[1]
    rescale = np.array([scale_x, scale_y, 1])
    
    
    mmpt = pd.read_csv(
        "../Merlin_output/images/micron_to_mosaic_pixel_transform.csv", 
        header=None, 
        sep=' '
    )
    mmpt = np.array(mmpt)

    def rescale_fun(x,y,z):
        points = np.array([x,y,z])
        # coordinates to pixel
        points = np.diag(mmpt) * points.T + np.append(mmpt[0:2,2], 0)
        # apply our scaling
        points = points / rescale
        return(points.T)
    
    return(rescale_fun)
rescale_fun = create_rescaling_function()
rescale_fun

<function __main__.create_rescaling_function.<locals>.rescale_fun(x, y, z)>

There are a few **Blank** genes that we can remove. We do NOT want to use those for cell segmentation

In [9]:
transcripts = transcripts[~transcripts['gene'].str.startswith("Blank")].copy()

In [ ]:
points = rescale_fun(transcripts.global_x, transcripts.global_y, transcripts.global_z)
# round to integer that we can use it with the cell segmentation mask
points = np.round(points).astype(int)
points

array([[14852, 15042, 15321, ...,  8689,  9156,  8899],
       [  411,   395,   364, ..., 30153, 30075, 29967],
       [    1,     3,     0, ...,     0,     0,     0]])

Next, we can import the segmentation mask from the previous script.

In [10]:
masks = np.load("cellpose_segmentation/masks.npy")

And we can add the cellpose_segmentation to the dataframe.

In [11]:
transcripts['mask'] = masks[
  tuple(points[[2,1,0]])
]

In [12]:
transcripts

,global_x,global_y,global_z,gene,mask
0,3822.614000,134.603240,4.0,Pycr1,0
1,3865.508500,89.455734,5.0,Slc7a11,0
2,3972.906500,163.452200,6.0,Cxcr3,0
3,3968.895800,33.911680,0.0,Ldhb,0
5,3664.242700,184.795300,4.0,Slc2a2,0
...,...,...,...,...,...
17812303,3266.297176,6421.473477,6.0,B2m,0
17812304,3499.033270,6353.866514,6.0,B2m,136929
17812305,3502.489212,6355.486490,6.0,B2m,54100
17812306,3848.083411,6330.754869,6.0,B2m,0


Lets see how many transcripts are in a cell. 

In [13]:
np.sum(transcripts['mask'] > 0) / transcripts.shape[0]

0.44092432073869814

We can now save the new transcripts.csv file and use that as an input for baysor

In [14]:
transcripts.to_csv("cellpose_segmentation/transcripts.csv")

In [15]:
np.max(masks)

137024